In [1]:
import sys
sys.path.append('..')
%matplotlib inline

#Convergence speed comparison according to Success Threshold parameter

The Success threshold Strategy depends on a meta-parameter, the value of the threshold. We will here compare the convergence status (defined by the information theoretic measure introduced in the paper) against this threshold parameter, at different times.

In [2]:
from lib import ngdb
import numpy as np
from additional import custom_graph
import seaborn as sns

In [3]:
db=ngdb.NamingGamesDB("graph2.db")

Here the configuration depends on the parameter:

In [4]:
def xp_cfg_thresh(i):
    return {
    'pop_cfg':{
        'voc_cfg':{
            'voc_type':'sparse_matrix',
            'M':20,
            'W':20
            },
        'strat_cfg':{
            'strat_type':'success_threshold',
            'threshold_explo':i
            },
        'nbagent':20
        },
    'step':500
    }

In [5]:
nb_iter = 8
values = np.linspace(0,1,21)
Tmax = [500, 1000, 2000, 10000]
measure = 'entropycouples_norm'

We generate a dict object of experiments

In [ ]:
exp_dict = {}
for v in values:
    exp_dict[v]=[]
    uuid_list=[]
    for i in range(nb_iter):
        tempexp = db.get_experiment(blacklist=uuid_list, tmax=Tmax[-1], **xp_cfg_thresh(v))
        exp_dict[v].append(tempexp)
        uuid_list.append(tempexp.uuid)

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


We define the procedure to graph what we want, and test it for the first value of T:

In [ ]:
def get_graph(T):
    Ytemp = []
    stdtemp = []
    ind = 0
    Ttemp=[0]
    for v in values:
        templist = []
        for i in range(nb_iter):
            tempexp = exp_dict[v][i]
            tempexp.continue_exp_until(T)
            if ind == 0:
                Ttemp = exp_dict[values[0]][0]._T
                while T > Ttemp[ind]:
                    ind+=1
            templist.append(tempexp.graph(measure)._Y[0][ind])
        Ytemp.append(np.mean(templist))
        stdtemp.append(np.std(templist))
    return custom_graph.CustomGraph(values, Ytemp, stdvec=stdtemp)

graph = get_graph(Tmax[0])
graph.std = 1
graph.show()

We then graph the other values, combine them to the previous graph, and plot everything:

In [ ]:
for T in Tmax[1:]: 
    graph.add_graph(get_graph(T))
graph.show()

In [ ]:
graph._Y = list(reversed(graph._Y)) # Execute just once. This is for presentation style only (see effect on legend).
graph.stdvec = list(reversed(graph.stdvec))
Tmax = list(reversed(Tmax))

In [ ]:
graph.title = ''
graph.xlabel = 'Threshold Parameter'
graph.ylabel = 'Normalized Information'
graph.ymin = [1,0]
graph.ymax = [1,1]

for i in range(len(graph.Yoptions)):
    graph.Yoptions[i] = {'label':'T={}'.format(str(Tmax[i]))}
    
graph.Yoptions[0]['color'] = 'black'
graph.Yoptions[1]['color'] = sns.color_palette()[1]
graph.Yoptions[2]['color'] = sns.color_palette()[2]
#graph.Yoptions[3]['color'] = sns.color_palette()[0]
    
graph.legendoptions['bbox_to_anchor'] = (0,0,0.5,1)

graph.show()

Convergence speed comparison according to threshold parameter, for the success-threshold strategy. The previously selected value of 90\% is well-chosen. $M$=$W$=$N$=20, 8 iterations. Time unit is number of past interactions.